In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from scrape_utils import WebScrapingUtilities
%run ../load_magic/storage.py
from ha_utils import HeaderAnalysis
from cypher_utils import CypherUtilities

wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
s = Storage()
ha = HeaderAnalysis()
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

CPU times: total: 1.94 s
Wall time: 2.6 s


In [4]:

%run ../load_magic/dataframes.py
from IPython.display import clear_output

In [5]:

files_list = sorted([fn for fn in os.listdir(cu.SAVES_HTML_FOLDER) if fn.endswith('.html')])
verbose = True
cu.ensure_navigableparent('END', verbose=verbose)
for file_name in files_list:
    cu.ensure_filename(file_name, verbose=verbose)
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        for sequence_order, (navigable_parent1, navigable_parent2) in enumerate(zip(child_strs_list[:-1], child_strs_list[1:])):
            clear_output(wait=True)
            cu.ensure_navigableparent(navigable_parent1, verbose=verbose)
            cu.ensure_navigableparents_relationship(navigable_parent1, navigable_parent2, file_name, sequence_order, verbose=verbose)
        
        # Add a fake relationship at the end
        clear_output(wait=True)
        cu.ensure_navigableparent(navigable_parent2, verbose=verbose)
        cu.ensure_navigableparents_relationship(navigable_parent2, 'END', file_name, sequence_order+1, verbose=verbose)
        
        child_tags_list = ha.get_child_tags_list(child_strs_list)
        for sequence_order, (navigable_parent, header_tag) in enumerate(zip(child_strs_list, child_tags_list)):
            clear_output(wait=True)
            header_tag_id = cu.get_headertag_id(header_tag, verbose=verbose)
            header_tag_sequence_id = cu.get_headertagsequence_id(file_name, header_tag_id, sequence_order, verbose=verbose)
            cu.ensure_headertagsequence_filename_relationship(file_name, verbose=verbose)
            cu.ensure_headertagsequence_headertag_relationship(header_tag_id, verbose=verbose)
            
            cu.ensure_headertag_navigableparent_relationship(header_tag_id, navigable_parent, verbose=verbose)

MERGE (hts:HeaderTagSequence {
                file_name: "e94e389195155d43_Staff_Software_Engineer_Seattle_WA_98102_Indeed_com.html",
                header_tag_id: "90",
                sequence_order: "32"})
            RETURN hts.header_tag_sequence_id;

                MERGE (hts:HeaderTagSequence {
                    file_name: "e94e389195155d43_Staff_Software_Engineer_Seattle_WA_98102_Indeed_com.html",
                    header_tag_id: "90",
                    sequence_order: "32"}) SET
                    hts.header_tag_sequence_id = "588ab023104dd183eb899d3330f2a190"
                RETURN hts.header_tag_sequence_id;
MATCH (fn:FileNames {file_name: "e94e389195155d43_Staff_Software_Engineer_Seattle_WA_98102_Indeed_com.html"}), (hts:HeaderTagSequence {file_name: "e94e389195155d43_Staff_Software_Engineer_Seattle_WA_98102_Indeed_com.html"})
            MERGE (hts)-[r:IS_CONTAINED_IN]->(fn);
MATCH
            (ht:HeaderTags {header_tag_id: "90"}),
            (hts:HeaderTagSeque

In [6]:

verbose = True
file_name = '9dc7201db74605e8_Data_Business_Analyst_Remote_Indeed_com.html'
file_name_id = cu.get_filename_id(file_name, verbose=verbose)
file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
page_soup = get_page_soup(file_path)
row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
for div_soup in row_div_list:
    child_strs_list = ha.get_navigable_children(div_soup, [])
    child_tags_list = ha.get_child_tags_list(child_strs_list)
    for sequence_order, (navigable_parent, header_tag) in enumerate(zip(child_strs_list, child_tags_list)):
        clear_output(wait=True)
        header_tag_id = cu.get_headertag_id(header_tag, verbose=verbose)
        header_tag_sequence_id = cu.get_headertagsequence_id(file_name_id, header_tag_id, sequence_order, verbose=verbose)
        cu.ensure_headertagsequence_filename_relationship(file_name_id, verbose=verbose)
        cu.ensure_headertagsequence_headertag_relationship(header_tag_id, verbose=verbose)

        navigable_parent_id = cu.get_navigableparent_id(navigable_parent, verbose=verbose)
        cu.ensure_headertag_navigableparent_relationship(header_tag_id, navigable_parent_id, verbose=verbose)
    for sequence_order, (navigable_parent1, navigable_parent2) in enumerate(zip(child_strs_list[:-1], child_strs_list[1:])):
        cu.ensure_navigableparent(navigable_parent1, verbose=verbose)
        cu.ensure_navigableparent(navigable_parent2, verbose=verbose)
        cu.ensure_navigableparents_relationship(navigable_parent1, navigable_parent2, file_name, sequence_order, verbose=verbose)

    # Add a fake relationship at the end
    cu.ensure_navigableparent('END', verbose=verbose)
    cu.ensure_navigableparents_relationship(navigable_parent2, 'END', file_name, sequence_order+1, verbose=verbose)

MERGE (hts:HeaderTagSequence {
                file_name: "7241b5e76974b36bb5614da42aa52f9d",
                header_tag_id: "88",
                sequence_order: "0"})
            RETURN hts.header_tag_sequence_id;

                MERGE (hts:HeaderTagSequence {
                    file_name: "7241b5e76974b36bb5614da42aa52f9d",
                    header_tag_id: "88",
                    sequence_order: "0"}) SET
                    hts.header_tag_sequence_id = "54850e003886e7b2fcc08719110bb868"
                RETURN hts.header_tag_sequence_id;
MATCH (fn:FileNames {file_name: "7241b5e76974b36bb5614da42aa52f9d"}), (hts:HeaderTagSequence {file_name: "7241b5e76974b36bb5614da42aa52f9d"})
            MERGE (hts)-[r:IS_CONTAINED_IN]->(fn);
MATCH
            (ht:HeaderTags {header_tag_id: "88"}),
            (hts:HeaderTagSequence {header_tag_id: "88"})
            MERGE (ht)-[r:IS_PART_OF]->(hts);


AttributeError: 'CypherUtilities' object has no attribute 'get_navigableparent_id'